<a href="https://colab.research.google.com/github/Tothu101/CNN-homework/blob/main/food_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon May  5 07:11:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

--2025-05-05 07:16:52--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.13K  --.-KB/s    in 0s      

2025-05-05 07:16:52 (51.9 MB/s) - ‘/content/train_val_split.py’ saved [3203/3203]

Created folder at /content/data/train/images.
Created folder at /content/data/train/labels.
Created folder at /content/data/validation/images.
Created folder at /content/data/validation/labels.
Number of image files: 127
Number of annotation files: 127
Images moving to train: 114
Images moving to validation: 13


In [ ]:
!pip install ultralytics

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 13
names:
- Ca sot ca
- Canh bi do
- Canh ca
- Canh chua
- Chocopie
- Com
- Dau bap luoc
- Dau hu nhoi thit
- Ga kho
- Rau luoc
- Rau xao thit
- Thit rang tom
- Trung cuon


In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, 

In [ ]:
!yolo detect predict model=/content/runs/detect/train4/weights/best.pt source=/content/z6569456504565_299a8c162da1510b89b827f859850ac9.jpg save=True

Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,417,831 parameters, 0 gradients, 21.3 GFLOPs

image 1/1 /content/z6569456504565_299a8c162da1510b89b827f859850ac9.jpg: 480x640 1 Chocopie, 1 Com, 1 Dau hu nhoi thit, 1 Ga kho, 47.4ms
Speed: 3.8ms preprocess, 47.4ms inference, 167.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
import json
price_map = {
    'Ca sot ca': 50000,
    'Canh bi do': 30000,
    'Canh ca': 35000,
    'Canh chua': 25000,
    'Chocopie': 15000,
    'Com': 20000,
    'Dau bap luoc': 10000,
    'Dau hu nhoi thit': 30000,
    'Ga kho': 45000,
    'Rau luoc': 10000,
    'Rau xao thit': 25000,
    'Thit rang tom': 40000,
    'Trung cuon': 20000
}


# Ghi vào file JSON
with open("price_list.json", "w", encoding='utf-8') as f:
    json.dump(price_map, f, ensure_ascii=False, indent=2)



In [ ]:
import torch
import json
import cv2
import numpy as np

# === Load model YOLO đã detect sẵn các món ăn ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/runs/detect/train4/weights/best.pt')  # chỉnh lại path mô hình
model.conf = 0.4

# === Load bảng giá từ file JSON ===
with open('/content/price_list.json', 'r', encoding='utf-8') as f:
    price_dict = json.load(f)

# === Nhận diện món ăn từ ảnh khay cơm ===
img_path = '/content/runs/detect/predict2/z6569456504565_299a8c162da1510b89b827f859850ac9.jpg'  # chỉnh đúng path ảnh bạn vừa test
image = cv2.imread(img_path)
results = model(img_path)

# === Lấy kết quả dự đoán từ YOLO ===
detections = results.pandas().xyxy[0]  # Tọa độ bounding box và tên lớp
labels = detections['name'].tolist()  # Lấy tên món ăn (class names)
boxes = detections[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()  # Tọa độ bounding box của món ăn

# === Tính tiền cho từng món ăn ===
total = 0
print("🧾 HÓA ĐƠN KHAY CƠM:")
for label, box in zip(labels, boxes):
    # Lấy giá của món ăn từ bảng giá, nếu không có thì báo lỗi và bỏ qua
    price = price_dict.get(label, None)
    if price is None:
        print(f"⚠️ Không có giá cho món {label}. Đã bỏ qua món này.")
        continue
    print(f" - {label} (Box: {box}): {price:,} VND")
    total += price

    # Vẽ bounding box lên ảnh
    xmin, ymin, xmax, ymax = map(int, box)
    color = (0, 255, 0)  # Màu xanh cho bounding box
    thickness = 2
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, thickness)

    # Thêm tên món ăn và giá vào ảnh
    label_price_text = f"{label}: {price:,} VND"
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, label_price_text, (xmin, ymin - 10), font, 0.6, (0, 255, 0), 2)

# In ra tổng tiền
print(f"\n💰 TỔNG TIỀN: {total:,} VND")

# === Lưu và hiển thị ảnh với bounding boxes ===
output_image_path = "/content/annotated_image.jpg"
cv2.imwrite(output_image_path, image)
print(f"📸 Hình ảnh đã được lưu tại {output_image_path}")

# Hiển thị hình ảnh (nếu cần)
# cv2.imshow("Annotated Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-5 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)



YOLO11s summary (fused): 100 layers, 9,417,831 parameters, 0 gradients, 21.3 GFLOPs


Adding AutoShape... 


Exception: 'Detect' object has no attribute 'grid'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.